# Metatrader 5
___

### Initialize and Login

In [14]:
# %pip install MetaTrader5
# %pip install pandas
# %pip install plotly
# %pip install python-dotenv


In [15]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px

# To access .env file
from dotenv import load_dotenv

import os
import time

In [16]:
# Access .env file
load_dotenv(".env")

# Access variables directly
username = int(os.getenv("USER_NAME"))
password = os.getenv("PASSWORD")
server = os.getenv("SERVER")

symbols = os.getenv("SYMBOLS")

In [17]:
# Initialize and log in to MetaTrader 5
# connect to MetaTrader 5
try:
    # Initialize MetaTrader 5
    if mt5.initialize():
        print("MetaTrader successfully initialized")
        
        # Log in to MetaTrader 5
        if mt5.login(login=username, password=password, server=server):
            print("MetaTrader login successful")
                
        else:
            print("MetaTrader login failed")
            mt5.shutdown()  # Ensure shutdown if login fails
    else:
        print("Failed to initialize MetaTrader 5")
except Exception as e:
    print(f"An error occurred: {e}")

MetaTrader successfully initialized
MetaTrader login successful


### bot

#### Initial code (don't use)

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import talib
import time
import logging
from datetime import datetime

# Initialize MT5 connection
if not mt5.initialize():
    print("MT5 initialization failed")
    quit()

# Configure logging
logging.basicConfig(
    filename="trade_bot.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("Trading bot started.")

def fetch_data(symbol, timeframe, num_candles=500):
    """
    Fetches recent market data for the specified symbol and timeframe.
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles)
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    return data

def execute_trade(symbol, action, lot_size):
    """
    Executes a trade on MT5, accounting for spread.
    """
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        logging.error(f"Symbol {symbol} not found.")
        return

    if not symbol_info.visible:
        logging.warning(f"Symbol {symbol} is not visible. Attempting to make it visible...")
        if not mt5.symbol_select(symbol, True):
            logging.error(f"Failed to select symbol {symbol}.")
            return

    point = symbol_info.point
    spread = symbol_info.spread * point  # Calculate the spread in price terms
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid
    half_spread = spread / 2  # Half spread for adjustment

    if action == "buy":
        entry_price = price + half_spread
        sl = entry_price - 0.0001  # Example: 1 pip below entry price
        tp = entry_price + 0.0005  # Example: 5 pips above entry price
    else:  # sell
        entry_price = price - half_spread
        sl = entry_price + 0.0001  # Example: 1 pip above entry price
        tp = entry_price - 0.0005  # Example: 5 pips below entry price

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_BUY if action == "buy" else mt5.ORDER_SELL,
        "price": entry_price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "MA Crossover Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Trade execution failed: {result.comment}")
    else:
        logging.info(f"{action.capitalize()} trade executed at {entry_price}. SL: {sl}, TP: {tp}")

def trade_logic(symbol, lot_size, num1=12, num2=13):
    """
    Implements the moving average crossover strategy.
    """
    # Fetch data
    data = fetch_data(symbol, mt5.TIMEFRAME_H2)

    # Calculate moving averages
    data['MA_12'] = talib.SMA(data['close'], timeperiod=num1)
    data['MA_13'] = talib.SMA(data['close'], timeperiod=num2)

    # Check for crossover signals
    if data['MA_12'].iloc[-1] > data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] <= data['MA_13'].iloc[-2]:
        logging.info(f"Buy signal detected for {symbol}.")
        execute_trade(symbol, "buy", lot_size)
    elif data['MA_12'].iloc[-1] < data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] >= data['MA_13'].iloc[-2]:
        logging.info(f"Sell signal detected for {symbol}.")
        execute_trade(symbol, "sell", lot_size)
    else:
        logging.info("No trade signal detected.")

# Controlled Execution with While Loop
symbol = "EURUSDm"  # Replace with your trading symbol
lot_size = 0.1  # Adjust based on your risk management

try:
    while True:
        trade_logic(symbol, lot_size)
        logging.info(f"Checked at {datetime.now()} - Waiting for the next cycle...")
        time.sleep(3600)  # Wait for 1 hour (H2 timeframe) before checking again
except KeyboardInterrupt:
    logging.info("Bot stopped by user.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
finally:
    mt5.shutdown()
    logging.info("MT5 connection closed.")


#### With calculated sleet time (don't use)

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import talib
import time
import logging
from datetime import datetime

# Initialize MT5 connection
if not mt5.initialize():
    print("MT5 initialization failed")
    quit()

# Configure logging
logging.basicConfig(
    filename="trade_bot.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("Trading bot started.")

def fetch_data(symbol, timeframe, num_candles=500):
    """
    Fetches recent market data for the specified symbol and timeframe.
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles)
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    return data

def execute_trade(symbol, action, lot_size):
    """
    Executes a trade on MT5, accounting for spread.
    """
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        logging.error(f"Symbol {symbol} not found.")
        return

    if not symbol_info.visible:
        logging.warning(f"Symbol {symbol} is not visible. Attempting to make it visible...")
        if not mt5.symbol_select(symbol, True):
            logging.error(f"Failed to select symbol {symbol}.")
            return

    point = symbol_info.point
    spread = symbol_info.spread * point  # Calculate the spread in price terms
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid
    half_spread = spread / 2  # Half spread for adjustment

    if action == "buy":
        entry_price = price + half_spread
        sl = entry_price - 0.0001  # Example: 1 pip below entry price
        tp = entry_price + 0.0005  # Example: 5 pips above entry price
    else:  # sell
        entry_price = price - half_spread
        sl = entry_price + 0.0001  # Example: 1 pip above entry price
        tp = entry_price - 0.0005  # Example: 5 pips below entry price

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_BUY if action == "buy" else mt5.ORDER_SELL,
        "price": entry_price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "MA Crossover Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Trade execution failed: {result.comment}")
    else:
        logging.info(f"{action.capitalize()} trade executed at {entry_price}. SL: {sl}, TP: {tp}")

def trade_logic(symbol, lot_size, num1=12, num2=13):
    """
    Implements the moving average crossover strategy.
    """
    # Fetch data
    data = fetch_data(symbol, mt5.TIMEFRAME_H2)

    # Calculate moving averages
    data['MA_12'] = talib.SMA(data['close'], timeperiod=num1)
    data['MA_13'] = talib.SMA(data['close'], timeperiod=num2)

    # Check for crossover signals
    if data['MA_12'].iloc[-1] > data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] <= data['MA_13'].iloc[-2]:
        logging.info(f"Buy signal detected for {symbol}.")
        execute_trade(symbol, "buy", lot_size)
    elif data['MA_12'].iloc[-1] < data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] >= data['MA_13'].iloc[-2]:
        logging.info(f"Sell signal detected for {symbol}.")
        execute_trade(symbol, "sell", lot_size)
    else:
        logging.info("No trade signal detected.")

def calculate_sleep_time():
    """
    Calculates the time to sleep until the next candlestick is complete.
    For an H2 timeframe, this ensures alignment with 2-hour intervals.
    """
    now = datetime.now()
    minutes_to_next_candle = (120 - (now.minute % 120)) - now.minute % 60
    seconds_to_next_candle = (minutes_to_next_candle * 60) - now.second
    return seconds_to_next_candle

# Controlled Execution with While Loop
symbol = "EURUSDm"  # Replace with your trading symbol
lot_size = 0.1  # Adjust based on your risk management

try:
    while True:
        trade_logic(symbol, lot_size)
        logging.info(f"Checked at {datetime.now()} - Waiting for the next cycle...")
        sleep_time = calculate_sleep_time()
        logging.info(f"Sleeping for {sleep_time} seconds.")
        time.sleep(sleep_time)
except KeyboardInterrupt:
    logging.info("Bot stopped by user.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
finally:
    mt5.shutdown()
    logging.info("MT5 connection closed.")


#### With calculated sleet time  and data recording feature ___(Use This)___

##### 2H EURUSDm

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import talib
import time
import logging
from datetime import datetime

# Initialize MT5 connection
if not mt5.initialize():
    print("MT5 initialization failed")
    quit()

# Configure logging
logging.basicConfig(
    filename="trade_bot.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("Trading bot started.")

# Initialize trade records DataFrame
trade_records = pd.DataFrame(columns=["symbol", "action", "entry_price", "sl", "tp", "result", "timestamp"])
trade_records_file = "trade_records.csv"

def fetch_data(symbol, timeframe, num_candles=500):
    """
    Fetches recent market data for the specified symbol and timeframe.
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles)
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    return data

def execute_trade(symbol, action, lot_size, timeframe):
    """
    Executes a trade on MT5, accounting for spread.
    Records the trade in a CSV file with the result.
    """
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        logging.error(f"Symbol {symbol} not found.")
        return

    if not symbol_info.visible:
        logging.warning(f"Symbol {symbol} is not visible. Attempting to make it visible...")
        if not mt5.symbol_select(symbol, True):
            logging.error(f"Failed to select symbol {symbol}.")
            return

    point = symbol_info.point
    spread = symbol_info.spread * point  # Calculate the spread in price terms
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid

    if action == "buy":
        entry_price = price + spread
        sl = entry_price - (1 * symbol_info.point * 10)  # 1 pip below entry
        tp = entry_price + (5 * symbol_info.point * 10)  # 5 pips above entry
    else:  # sell
        entry_price = price - spread
        sl = entry_price + (1 * symbol_info.point * 10)  # 1 pip above entry
        tp = entry_price - (5 * symbol_info.point * 10)  # 5 pips below entry


    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_TYPE_BUY if action == "buy" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "MA Crossover Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Trade execution failed: {result.comment}")
    else:
        logging.info(f"{action.capitalize()} trade executed at {entry_price}. SL: {sl}, TP: {tp}")

        # Wait for trade to close and determine the result
        trade_result = "Pending"
        while trade_result == "Pending":
            positions = mt5.positions_get(symbol=symbol)
            if not positions:
                break
            trade = positions[0]
            if trade.profit >= tp:
                trade_result = "Success"
            elif trade.profit <= sl:
                trade_result = "Failure"
            time.sleep(5)

        # Record trade in CSV
        trade_record = {
            "symbol": symbol,
            "action": action,
            "entry_price": entry_price,
            "sl": sl,
            "tp": tp,
            "result": trade_result,
            "timestamp": datetime.now(),
        }
        global trade_records
        trade_records = trade_records.append(trade_record, ignore_index=True)
        trade_records.to_csv(trade_records_file, index=False)
        logging.info(f"Trade recorded: {trade_record}")

def trade_logic(symbol, lot_size, timeframe, num1=12, num2=13):
    """
    Implements the moving average crossover strategy.
    """
    # Fetch data
    data = fetch_data(symbol, timeframe)

    # Calculate moving averages
    data['MA_12'] = talib.SMA(data['close'], timeperiod=num1)
    data['MA_13'] = talib.SMA(data['close'], timeperiod=num2)

    # Check for crossover signals
    if data['MA_12'].iloc[-1] > data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] <= data['MA_13'].iloc[-2]:
        logging.info(f"Buy signal detected for {symbol}.")
        execute_trade(symbol, "buy", lot_size, timeframe)
    elif data['MA_12'].iloc[-1] < data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] >= data['MA_13'].iloc[-2]:
        logging.info(f"Sell signal detected for {symbol}.")
        execute_trade(symbol, "sell", lot_size, timeframe)
    else:
        logging.info("No trade signal detected.")

def calculate_sleep_time():
    """
    Calculates the time to sleep until the next candlestick is complete.
    For an H2 timeframe, this ensures alignment with 2 hour intervals.
    """
    now = datetime.now()
    minutes_to_next_candle = (10 - (now.minute % 10))
    seconds_to_next_candle = (minutes_to_next_candle * 60) - now.second
    return seconds_to_next_candle


# Controlled Execution with While Loop
symbol = "EURUSDm"  
lot_size = 0.1  
timeframe = mt5.TIMEFRAME_H2  

try:
    while True:
        trade_logic(symbol, lot_size, timeframe)
        logging.info(f"Checked at {datetime.now()} - Waiting for the next cycle...")
        sleep_time = calculate_sleep_time()
        logging.info(f"Sleeping for {sleep_time/120} hours.")
        time.sleep(sleep_time)
except KeyboardInterrupt:
    logging.info("Bot stopped by user.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
finally:
    mt5.shutdown()
    logging.info("MT5 connection closed.")


##### 10M BTCUSDm

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import talib
import time
import logging
from datetime import datetime

# Initialize MT5 connection
if not mt5.initialize():
    print("MT5 initialization failed")
    quit()

# Configure logging
logging.basicConfig(
    filename="trade_bot.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("Trading bot started.")

# Initialize trade records DataFrame
trade_records = pd.DataFrame(columns=["symbol", "action", "entry_price", "sl", "tp", "result", "timestamp"])
trade_records_file = "trade_records.csv"

def fetch_data(symbol, timeframe, num_candles=500):
    """
    Fetches recent market data for the specified symbol and timeframe.
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles)
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    return data

def execute_trade(symbol, action, lot_size, timeframe):
    """
    Executes a trade on MT5, accounting for spread.
    Records the trade in a CSV file with the result.
    """
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        logging.error(f"Symbol {symbol} not found.")
        return

    if not symbol_info.visible:
        logging.warning(f"Symbol {symbol} is not visible. Attempting to make it visible...")
        if not mt5.symbol_select(symbol, True):
            logging.error(f"Failed to select symbol {symbol}.")
            return
    # The point and pip are the same in the "BTCUSDm" pair = 0.01
    point = symbol_info.point
    spread = symbol_info.spread * point  # Calculate the spread in price terms
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid

    if action == "buy":
        entry_price = price + spread
        sl = entry_price - (1 * symbol_info.point)  # 1 pip below entry
        tp = entry_price + (5 * symbol_info.point)  # 5 pips above entry
    else:  # sell
        entry_price = price - spread
        sl = entry_price + (1 * symbol_info.point)  # 1 pip above entry
        tp = entry_price - (5 * symbol_info.point)  # 5 pips below entry


    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_TYPE_BUY if action == "buy" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "MA Crossover Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Trade execution failed: {result.comment}")
    else:
        logging.info(f"{action.capitalize()} trade executed at {entry_price}. SL: {sl}, TP: {tp}")

        # Wait for trade to close and determine the result
        trade_result = "Pending"
        while trade_result == "Pending":
            positions = mt5.positions_get(symbol=symbol)
            if not positions:
                break
            trade = positions[0]
            if trade.profit >= tp:
                trade_result = "Success"
            elif trade.profit <= sl:
                trade_result = "Failure"
            time.sleep(5)

        # Record trade in CSV
        trade_record = {
            "symbol": symbol,
            "action": action,
            "entry_price": entry_price,
            "sl": sl,
            "tp": tp,
            "result": trade_result,
            "timestamp": datetime.now(),
        }
        global trade_records
        trade_records = trade_records.append(trade_record, ignore_index=True)
        trade_records.to_csv(trade_records_file, index=False)
        logging.info(f"Trade recorded: {trade_record}")

def trade_logic(symbol, lot_size, timeframe, num1=12, num2=13):
    """
    Implements the moving average crossover strategy.
    """
    # Fetch data
    data = fetch_data(symbol, timeframe)

    # Calculate moving averages
    data['MA_12'] = talib.SMA(data['close'], timeperiod=num1)
    data['MA_13'] = talib.SMA(data['close'], timeperiod=num2)

    # Check for crossover signals
    if data['MA_12'].iloc[-1] > data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] <= data['MA_13'].iloc[-2]:
        logging.info(f"Buy signal detected for {symbol}.")
        execute_trade(symbol, "buy", lot_size, timeframe)
    elif data['MA_12'].iloc[-1] < data['MA_13'].iloc[-1] and data['MA_12'].iloc[-2] >= data['MA_13'].iloc[-2]:
        logging.info(f"Sell signal detected for {symbol}.")
        execute_trade(symbol, "sell", lot_size, timeframe)
    else:
        logging.info("No trade signal detected.")

def calculate_sleep_time():
    """
    Calculates the time to sleep until the next candlestick is complete.
    For an M10 timeframe, this ensures alignment with 10min intervals.
    """
    now = datetime.now()
    minutes_to_next_candle = (10 - (now.minute % 10))
    seconds_to_next_candle = (minutes_to_next_candle * 60) - now.second
    return seconds_to_next_candle


# Controlled Execution with While Loop
symbol = "BTCUSDm"  
lot_size = 20  # "BTCUSDm" contract size is 1 with one pip size= 0.01 # max lot_size= "volume_max"
timeframe = mt5.TIMEFRAME_M10  

try:
    while True:
        trade_logic(symbol, lot_size, timeframe)
        logging.info(f"Checked at {datetime.now()} - Waiting for the next cycle...")
        sleep_time = calculate_sleep_time()
        logging.info(f"Sleeping for {sleep_time/60} minutes.")
        time.sleep(sleep_time)
except KeyboardInterrupt:
    logging.info("Bot stopped by user.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
finally:
    mt5.shutdown()
    logging.info("MT5 connection closed.")
